In [ ]:
#Script pour exécuter des commandes shell et créer des csv à partir de requêtes SQL dans un conteneur Docker SQLite
import subprocess

def run_command(command, shell=False):
    """Helper to run shell or list-based commands with error handling."""
    try:
        result = subprocess.run(command, shell=shell, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {command}")
        print(e.stderr)



In [3]:
#Copy the SQLite DB into the container
print("📁 Copying newIMDB.db into docker container...")
run_command(["docker", "cp", "../data/db/newIMDB.db", "moviesdb_sqlite:/data/newIMDB.db"])



📁 Copying newIMDB.db into docker container...



In [9]:
#Start the container using the shell script
print("🚀 Starting Docker container with start_sqlite.sh...")
COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
run_command(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


🚀 Starting Docker container with start_sqlite.sh...
#1 [internal] load local bake definitions
#1 reading from stdin 572B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.sqlite
#2 transferring dockerfile: 185B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/alpine:3.19
#3 ...

#4 [auth] library/alpine:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/alpine:3.19
#3 DONE 1.1s

#5 [internal] load .dockerignore
#5 transferring context: 277B done
#5 DONE 0.0s

#6 [1/3] FROM docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1
#6 resolve docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1 done
#6 DONE 0.0s

#7 [2/3] RUN apk add --no-cache sqlite bash
#7 CACHED

#8 [3/3] WORKDIR /data
#8 CACHED

#9 exporting to image
#9 exporting layers done
#9 exporting manifest sha256:c62068895fb5c5a190e6f8ddea60b4068

In [12]:
#copy the SQL file into the container
print("📁 Copying marvel_queries.sql into docker container...")
run_command(["docker", "cp", "../utils/marvel_queries.sql", "moviesdb_sqlite:/data/marvel_queries.sql"])


📁 Copying marvel_queries.sql into docker container...



In [13]:
#Run the SQL file using sqlite3 inside the container
print("📄 Executing marvel_queries.sql inside the container...")
run_command([
    "docker", "exec", "-i", "moviesdb_sqlite",
    "sqlite3", "/data/newIMDB.db", ".read /data/marvel_queries.sql"
])

📄 Executing marvel_queries.sql inside the container...



In [14]:
#copy csv file created by marvel_queries.sql from container to local
run_command([
    "docker", "cp", "moviesdb_sqlite:/data/marvel_movies.csv", "../data/tests/marvel_movies.csv"
])

In [16]:
run_command([
    "docker", "cp", "moviesdb_sqlite:/data/marvel_characters.csv", "../data/tests/marvel_characters.csv"
])

In [15]:
#read csv into pandas dataframe
import pandas as pd
marvel_df = pd.read_csv("../data/tests/marvel_movies.csv")
marvel_df.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0371746,Iron Man,2008,126.0,"Action,Adventure,Sci-Fi"
1,tt0486592,Guardians,2008,87.0,"Action,Adventure,Fantasy"
2,tt0800080,The Incredible Hulk,2008,112.0,"Action,Adventure,Sci-Fi"
3,tt0834967,A Viking Saga: Son of Thor,2008,82.0,"Adventure,Drama,History"
4,tt1690483,King of Thorn,2009,120.0,"Adventure,Animation,Horror"


In [17]:
#read csv into pandas dataframe
import pandas as pd
marvel_df = pd.read_csv("../data/tests/marvel_characters.csv")
marvel_df.head()

,character_name,actor_name,title_name
0,"[""Obadiah Stane""]",Jeff Bridges,Iron Man
1,"[""Tony Stark""]",Robert Downey Jr.,Iron Man
2,"[""Pepper Potts""]",Gwyneth Paltrow,Iron Man
3,"[""Christine Everhart""]",Leslie Bibb,Iron Man
4,"[""Rhodey""]",Terrence Howard,Iron Man


In [ ]:
#Stop and remove the container
print("🚀 Stopping Docker container with stop_sqlite.sh...")
COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
CONTAINER_NAME="moviesdb_sqlite"

run_command(["docker", "stop", CONTAINER_NAME])
run_command(["docker", "compose", "-f", COMPOSE_FILE, "down"])